In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

class Node:
    def __init__(self, x=None, y=None, hue=None, frequency=None):
        self.x = x
        self.y = y
        self.hue = hue
        self.frequency = frequency

# Define the area dimensions
area_width = 379
area_height = 379
grid_size = 1  # Set grid size to 1 to cover every possible (x, y) coordinate

# Generate the set of 8 nodes evenly spaced on the hue spectrum and mapped to the audible spectrum
hue_step = 360 / 8
frequency_range = (600, 600 * 8)  # Audible frequency range in Hz
frequency_step = 600  # Logarithmic spacing
nodes = [Node(hue=i * hue_step, frequency=frequency_range[0] + i * frequency_step) for i in range(8)]

# Create a grid of nodes covering the entire area
grid = [[random.choice(nodes) for _ in range(area_width // grid_size)] for _ in range(area_height // grid_size)]

# Flatten the grid into a list of nodes
all_nodes = [node for row in grid for node in row]

# Assign positions to nodes based on grid cell coordinates
for i, node in enumerate(all_nodes):
    node.x = (i % (area_width // grid_size)) * grid_size + random.uniform(0, grid_size)
    node.y = (i // (area_width // grid_size)) * grid_size + random.uniform(0, grid_size)

# Create lists of x and y coordinates, hues, and frequencies for plotting
x_coords = [node.x for node in all_nodes]
y_coords = [node.y for node in all_nodes]
hues = [node.hue for node in all_nodes]
frequencies = [node.frequency for node in all_nodes]

# Plot the nodes as a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(x_coords, y_coords, c=hues, cmap='hsv', alpha=0.8)
plt.colorbar(label='Hue')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.title('Node Distribution with Hues and Frequencies')
plt.grid(True)
plt.show()


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

class Node:
    def __init__(self, x=None, y=None, hue=None, frequency=None, layer=None):
        self.x = x
        self.y = y
        self.hue = hue
        self.frequency = frequency
        self.layer = layer
    
    def copy(self):
        return Node(self.x, self.y, self.hue, self.frequency, self.layer)
    

area_width = 379
area_height = 379
grid_size = 10  # Adjust the grid size as needed

# Generate the set of 8 nodes evenly spaced on the hue spectrum and mapped to the audible spectrum
hue_step = 360 / 8
frequency_range = (600, 600 * 8)  # Audible frequency range in Hz
frequency_step = 600  # Logarithmic spacing
nodes = [Node(hue=i * hue_step, frequency=frequency_range[0] + i * frequency_step) for i in range(8)]

for xx in range(0,378):
    for yy in range(0,378):
        node = random.choice(nodes).copy()
        node.x = xx
        node.y = yy
        node.layer = 0









In [ ]:
from deap import base, creator, tools, algorithms
import random

# Define individual and population
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("random_bit", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, list, toolbox.random_bit, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation function
def evaluate(individual):
    """Evaluate the fitness of an individual based on its value."""
    # Example fitness evaluation: simply return the value of the bit
    return individual[0],

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Evolutionary algorithm
# Evolutionary algorithm
def evolve(population, n_generations):
    for gen in range(n_generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        population = toolbox.select(offspring, k=len(population))
    return population

# Example usage
population = toolbox.population(n=100)
final_population = evolve(population, n_generations=50)


In [ ]:
from deap import base, creator, tools
import random
import numpy as np

# Game of Life utilities
def count_neighbors(grid, x, y):
    """Count the number of live neighbors around cell (x, y) in the grid."""
    count = 0
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            if 0 <= x + dx < len(grid) and 0 <= y + dy < len(grid[0]) and grid[x + dx][y + dy] == 1:
                count += 1
    return count

def evaluate(individual):
    """Evaluate the fitness of an individual based on its configuration in the Game of Life."""
    grid = np.array(individual).reshape(10, 10)
    new_grid = np.zeros_like(grid)
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            live_neighbors = count_neighbors(grid, i, j)
            if grid[i][j] == 1:  # Live cell
                if live_neighbors < 1:
                    new_grid[i][j] = 0  # Dies due to underpopulation
                elif live_neighbors == 1:
                    new_grid[i][j] = 1  # Survives and produces offspring
                elif live_neighbors == 2 or live_neighbors == 3:
                    new_grid[i][j] = 1  # Survives
                else:
                    new_grid[i][j] = 0  # Dies due to overpopulation
            else:  # Dead cell
                if live_neighbors == 3:
                    new_grid[i][j] = 1  # Reproduction
    return np.sum(new_grid),

# DEAP setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("random_bit", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, list, toolbox.random_bit, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Evolutionary algorithm
def evolve(population, n_generations):
    for gen in range(n_generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        population = toolbox.select(offspring, k=len(population))
    return population

# Example usage
population = toolbox.population(n=100)
final_population = evolve(population, n_generations=50)

In [27]:
import numpy as np
import pandas as pd

# Stability table with node types
stability_table = np.array([
    [-1,1, 2, 3, 4, 5, 6, 7, 8],
    [1, 1, 0, 0, 0, 0, 0, 0, 0],
    [2, 1, 1, 0, 1, 1, 1, 1, 1],
    [3, 1, 0, 1, 0, 1, 1, 0, 0],
    [4, 1, 0, 0, 1, 1, 1, 1, 0],
    [5, 1, 0, 0, 0, 1, 0, 0, 0],
    [6, 1, 0, 0, 0, 1, 1, 0, 0],
    [7, 1, 0, 0, 0, 1, 1, 1, 0],
    [8, 1, 0, 0, 1, 1, 1, 1, 1]
])

class Node:
    def __init__(self, top_type, bottom_type, stability, magnitude, pressure):
        self.top_type = top_type
        self.bottom_type = bottom_type
        self.stability = stability
        self.magnitude = magnitude
        self.pressure = pressure
    
    def __str__(self):
        return f"{self.top_type}{self.bottom_type}{self.stability};{self.magnitude};{self.pressure}"

# Function to generate a random individual (pairing)
def random_individual():
    # Choose a random index for "top" and "bottom" node types
    top_index = np.random.randint(1, 9)
    bottom_index = np.random.randint(1, 9)
    # Determine stability based on the stability table
    top_type = stability_table[top_index][0]
    bottom_type = stability_table[0][bottom_index]
    stability = stability_table[top_index][bottom_index]
    # Choose constant magnitude and pressure values
    magnitude = 1 
    pressure = 0
    # Create a Node object with the determined properties
    return Node(top_type, bottom_type, stability, magnitude, pressure)

# Function to initialize the grid with random pairings
def initialize_grid(size):
    grid = np.array([[random_individual() for _ in range(size)] for _ in range(size)])  # Generate random pairings for each cell
    return grid

# Initialize the grid
grid_size = 30
grid = initialize_grid(grid_size)

# Create DataFrame from the grid
df = pd.DataFrame(grid)

# Print the DataFrame
print(df)


         0        1        2        3        4        5        6        7   \
0   820;1;0  351;1;0  820;1;0  311;1;0  230;1;0  111;1;0  560;1;0  670;1;0   
1   540;1;0  661;1;0  620;1;0  871;1;0  551;1;0  830;1;0  651;1;0  130;1;0   
2   170;1;0  751;1;0  411;1;0  780;1;0  461;1;0  320;1;0  130;1;0  471;1;0   
3   670;1;0  680;1;0  540;1;0  820;1;0  221;1;0  881;1;0  241;1;0  771;1;0   
4   680;1;0  271;1;0  551;1;0  430;1;0  570;1;0  761;1;0  211;1;0  251;1;0   
5   180;1;0  261;1;0  120;1;0  661;1;0  261;1;0  520;1;0  160;1;0  430;1;0   
6   580;1;0  811;1;0  611;1;0  351;1;0  780;1;0  451;1;0  720;1;0  211;1;0   
7   441;1;0  420;1;0  311;1;0  661;1;0  620;1;0  511;1;0  340;1;0  150;1;0   
8   411;1;0  471;1;0  740;1;0  820;1;0  150;1;0  511;1;0  441;1;0  241;1;0   
9   320;1;0  311;1;0  580;1;0  640;1;0  861;1;0  441;1;0  730;1;0  331;1;0   
10  780;1;0  861;1;0  430;1;0  780;1;0  520;1;0  520;1;0  361;1;0  820;1;0   
11  351;1;0  830;1;0  241;1;0  150;1;0  221;1;0  451;1;0  830;1;